# Import fundmental ratio from FinViz

In [24]:
# For data manipulation
import pandas as pd

# To extract fundamental data
from bs4 import BeautifulSoup as bs
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
          'Upgrade-Insecure-Requests': '1', 'Cookie': 'v2=1495343816.182.19.234.142', 'Accept-Encoding': 'gzip, deflate, sdch',
           'Referer': "http://finviz.com/quote.ashx?t="}



**Create a function to parse data from FinViz **

In [28]:
def fundamental_metric(soup, metric):
    return soup.find(text=metric).find_next(class_='snapshot-td2').text

In [29]:
def get_fundamental_data(df):
    for symbol in df.index:
        try:
            url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
            r = requests.get("http://finviz.com/quote.ashx?t="+ symbol.lower(),headers=headers)
            soup = bs(r.content,'html.parser') 
            #soup = bs(requests.get(url).content) 
            for m in df.columns:                
                df.loc[symbol,m] = fundamental_metric(soup,m)                
        except Exception as e:
            print (symbol, 'not found')
            print(e)
    return df

**List of Stocks and Ratios You are Interested In**

In [32]:

stock_list = ['AMZN','GOOG','PG','KO','IBM','DG','XOM','KO','PEP','MT','NL','LPL']

metric = ['P/B',
'P/E',
'Forward P/E',
'PEG',
'Debt/Eq',
'EPS (ttm)',
'Dividend %',
'ROE',
'ROI',
'EPS Q/Q',
'Insider Own'
]

**Initialize Pandas DataFrame to Store the Data

In [33]:
df = pd.DataFrame(index=stock_list,columns=metric)
df = get_fundamental_data(df)
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,19.60,94.81,71.09,2.89,0.61,34.15,-,24.50%,11.70%,192.00%,10.60%
GOOG,5.93,36.00,30.28,2.14,-,51.75,-,-,-,59.10%,5.86%
PG,6.83,24.64,21.60,2.68,0.66,5.29,2.42%,29.40%,16.10%,4.30%,0.10%
KO,11.35,25.49,23.36,11.68,2.84,1.93,3.34%,45.50%,12.00%,-33.00%,0.70%
IBM,5.05,19.19,9.93,3.15,3.08,6.26,5.43%,38.20%,11.40%,-0.50%,0.10%
DG,7.04,19.72,19.81,1.17,0.00,10.09,0.72%,36.10%,18.90%,62.50%,0.10%
XOM,1.11,76.26,23.88,5.22,0.39,0.60,7.56%,1.80%,2.30%,-121.50%,0.20%
KO,11.35,25.49,23.36,11.68,2.84,1.93,3.34%,45.50%,12.00%,-33.00%,0.70%
PEP,14.29,27.49,22.99,4.49,3.31,5.06,2.94%,51.90%,17.80%,10.30%,0.10%
MT,0.77,-,6.89,-,0.37,-3.68,-,-10.60%,-2.10%,60.10%,44.80%


**Remove % Sign and Convert Values to Numeric Type¶ 

In [34]:
df['Dividend %'] = df['Dividend %'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROI'] = df['ROI'].str.replace('%', '')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%', '')
df['Insider Own'] = df['Insider Own'].str.replace('%', '')
df = df.apply(pd.to_numeric, errors='coerce')
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
AMZN,19.60,94.81,71.09,2.89,0.61,34.15,NaN,24.5,11.7,192.0,10.60
GOOG,5.93,36.00,30.28,2.14,NaN,51.75,NaN,NaN,NaN,59.1,5.86
PG,6.83,24.64,21.60,2.68,0.66,5.29,2.42,29.4,16.1,4.3,0.10
KO,11.35,25.49,23.36,11.68,2.84,1.93,3.34,45.5,12.0,-33.0,0.70
IBM,5.05,19.19,9.93,3.15,3.08,6.26,5.43,38.2,11.4,-0.5,0.10
DG,7.04,19.72,19.81,1.17,0.00,10.09,0.72,36.1,18.9,62.5,0.10
XOM,1.11,76.26,23.88,5.22,0.39,0.60,7.56,1.8,2.3,-121.5,0.20
KO,11.35,25.49,23.36,11.68,2.84,1.93,3.34,45.5,12.0,-33.0,0.70
PEP,14.29,27.49,22.99,4.49,3.31,5.06,2.94,51.9,17.8,10.3,0.10
MT,0.77,NaN,6.89,NaN,0.37,-3.68,NaN,-10.6,-2.1,60.1,44.80


## Filter Good Companies
**1. Companies which are quoted at low valuations**
P/E < 15 and P/B < 1

In [35]:
df_filtered = df[(df['P/E'].astype(float)<15) & (df['P/B'].astype(float) < 1)]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
NL,0.73,13.6,7.15,NaN,0.0,0.34,3.44,5.5,-0.7,351.6,83.0


**2. Further filter companies which have demonstrated earning power**
EPS Q/Q > 10%

In [36]:
df_filtered = df_filtered[df_filtered['EPS Q/Q'].astype(float) > 10]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
NL,0.73,13.6,7.15,NaN,0.0,0.34,3.44,5.5,-0.7,351.6,83.0


**Management having substantial ownership in the business**
Insider Own > 30%

In [37]:
df = df[df['Insider Own'].astype(float) > 30]
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own
MT,0.77,NaN,6.89,NaN,0.37,-3.68,NaN,-10.6,-2.1,60.1,44.8
NL,0.73,13.6,7.15,NaN,0.00,0.34,3.44,5.5,-0.7,351.6,83.0
LPL,0.73,NaN,16.11,NaN,1.32,-3.12,NaN,-22.1,-10.8,107.2,51.1
